# 

In [1]:
import dora
import pyarrow as pa
import inspect
import os
import numpy as np
import pandas as pd
import pylcs
import textwrap
import os
import sys
import time

In [2]:
node = dora.Node()

In [3]:


def search_most_simlar_line(text, searched_line):
    lines = text.split("\n")
    values = []

    for line in lines:
        values.append(pylcs.lcs_sequence_length(line, searched_line))
    output = lines[np.array(values).argmax()]
    return output


def strip_indentation(code_block):
    # Use textwrap.dedent to strip common leading whitespace
    dedented_code = textwrap.dedent(code_block)

    return dedented_code


def replace_code_with_indentation(original_code, replacement_code):
    # Split the original code into lines
    lines = original_code.splitlines()

    # Preserve the indentation of the first line
    indentation = lines[0][: len(lines[0]) - len(lines[0].lstrip())]

    # Create a new list of lines with the replacement code and preserved indentation
    new_code_lines = indentation + replacement_code

    return new_code_lines


def replace_source_code(source_code, gen_replacement):
    initial = search_most_simlar_line(source_code, gen_replacement)
    replacement = strip_indentation(gen_replacement.replace("```python\n", "").replace("\n```", "").replace("\n", ""))
    intermediate_result = replace_code_with_indentation(initial, replacement)
    end_result = source_code.replace(initial, intermediate_result)
    return end_result


def generate_code(node, prompt):
    ## Query VectorDB
    node.send_output("query", pa.array([prompt]))
    
    event = node.next(timeout=5)
    
    if event["type"] == "ERROR":
        raise (event["error"])
    else:
        source_code = event["value"]
        source_code = pd.DataFrame(data=source_code.to_pandas().to_numpy().reshape((-1, 3)), columns = ["path", "code", "on_input"])
    text = source_code["on_input"][0]
    
    ## Generate code based on retrieved data
    node.send_output("prompt", pa.array([f"{text} \n {prompt}. Respond with one line of code only "]))
    event = node.next(timeout=10)
    print(f"Generated code: ")
    if event["type"] == "ERROR":
        raise (event["error"])
    else:
        gen_replacement = event["value"][0].as_py()
        print(event["value"][0])

    # new source code
    new_source_code = replace_source_code(source_code["code"][0], gen_replacement)

    # Write to file
    with open(source_code["path"][0], 'w') as f:
        f.write(new_source_code)

In [4]:



## Get all python files path in given directory
def get_all_functions(path):
    functions = []
    paths = []
    codes = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".py"):
                path = os.path.join(root, file)
                with open(path, "r", encoding="utf8") as f:
                    ## add file folder to system path
                    sys.path.append(root)
                    ## import module from path
                    try:
                        module = __import__(file[:-3])
                        ## get all functions in module
                        for name, data in module.__dict__.items():
                            if inspect.isclass(data) and name == "Operator":
                                source = inspect.getsource(data.on_input)
                                codes.append(source)
                                functions.append(f.read())
                                paths.append(path)

                    except Exception as err:
                        print(err)
                        pass

    return functions, paths, codes

def generate_db(node, path):
    node.send_output("clear", pa.array([""]))
    files, paths, codes = get_all_functions(path)
    print(f"Number of files: {len(files)}")
    files.extend(paths)
    files.extend(codes)
    node.send_output("append", pa.array(files))
    
path = "/home/peter/Documents/work/dora-llm/operators/webcam/"


In [5]:
path = "/home/peter/Documents/work/dora-llm/operators/webcam/"
generate_db(node, path)

type object 'Operator' has no attribute 'on_input'
Number of files: 2


In [6]:
PROMPT = "Can you change the bounding box from rectangle to circles?"

In [7]:
generate_code(node, PROMPT)

Generated code: 
         cv2.circle(self.image, (int(min_x), int(min_y)), int(max_x - min_x), (0, 255, 0), 2)


In [1]:
node.send_output("speech", pa.array([""]))

trial: 0
trial: 1
trial: 2
trial: 3
trial: 4


In [11]:
node.send_output("speech", pa.array([""]))
time.sleep(7)
prompt = node.next(timeout=5)["value"][0].as_py()
if prompt is not None:
    generate_code(node, prompt)
else:
    print("did not work")

Generated code: 
         cv2.putText(self.image, LABELS[int(label)] + f", {confidence:0.2f}", (int(max_x), int(max_y)), font, 0.75, (0, 0, 0), 2, 1)


In [12]:
# If not ok revert!
with open(source_code["path"][0], 'w') as f:
    f.write(source_code["code"][0])

NameError: name 'source_code' is not defined

In [7]:
prompt

' Can you change the bounding box to blue?'